In [6]:
import re
import itertools
import numpy

def loadFasta(filename):
    if (filename.endswith(".gz")):
        fp = gzip.open(filename, 'rb')
    else:
        fp = open(filename, 'rb')
    # split at headers
    data = fp.read().split('>')
    fp.close()
    # ignore whatever appears before the 1st header
    data.pop(0)     
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        # add an extra "+" to make string "1-referenced"
        sequences.append('+' + ''.join(lines))
    return (headers, sequences)

def createBinaryVectorFile(kmerList):
    baseMatrices = {'A': numpy.asarray([0,0,0,1]).T, 'C': numpy.asarray([0,0,1,0]).T, 'G': numpy.asarray([0,1,0,0]).T, 'T': numpy.asarray([1,0,0,0]).T}
    binFile = open('kmerBinary.csv', 'w')
    for key, value in kmerList.iteritems():
        for kmer in value:
            for character in kmer:
                binFile.write(baseMatrices.get(character))
                binFile.write(",")
            binFile.write(key)
            binFile.write("\n")
    binFile.close()

def createHistogramFile(kmerList, histKmerLen):
    histFile = open('kmerHistograms.csv', 'w')
    motifs = {}
    motifList = []
    
    for pattern in itertools.product('ACTG', repeat=histKmerLen):
        motif = ''.join(pattern)
        motifs[motif] = 0
        motifList.append(motif)
        histFile.write(motif)
        histFile.write(",")
    
    histFile.write("\n")
    freqKmers = {}
    
    for key, value in kmerList.iteritems():
        for kmer in value:
            for i in xrange(len(kmer)-histKmerLen+1):
                histKmer = kmer[i:i+histKmerLen]
#                 motifFreq =  motifs.get(histKmer, 0)
#                 motifFreq += 1
                motifs[histKmer] = motifs.get(histKmer, 0) + 1
        maxFreq = 0
        maxKmer = ''

        for motif in motifList:
            if motifs[motif] > maxFreq:
                maxFreq = motifs[motif]
                maxKmer = motif
            histFile.write(str(motifs[motif]))
            histFile.write(",")
            motifs[motif] = 0
        freqKmers[maxKmer] = freqKmers.get(maxKmer, 0)+1
            
        histFile.write(key)
        histFile.write("\n")

#    print freqKmers
    histFile.close()
    
def callFasta(fileName, baseCoverage, longKmerLen, shortKmerLen, histKmerLen):
    header, seq = loadFasta(fileName)

    transcriptLocation = 1
    kmerList = {}
    for i in xrange(len(header)):
        transcriptID = re.split('\|+', header[i])[transcriptLocation][7:]
        kmerListForTranscript = []
#         for j in xrange(1, len(seq[i])-longKmerLen+1, (longKmerLen/baseCoverage)):
        for j in xrange(1, len(seq[i])-longKmerLen+1):
            kmer = seq[i][j:j+longKmerLen]
            for k in xrange(len(kmer)-shortKmerLen+1):
                shortKmer = kmer[k:k+shortKmerLen]
                kmerListForTranscript.append(shortKmer)

#         if (len(seq[i])-1) > longKmerLen and (len(seq[i])-1)%(longKmerLen/baseCoverage) != 0:
#             kmer = seq[i][-longKmerLen:]
#             for k in xrange(len(kmer)-shortKmerLen+1):
#                 shortKmer = kmer[k:k+shortKmerLen]
#                 kmerListForTranscript.append(shortKmer)
        if len(seq[i])-1 <= longKmerLen:
            kmer = seq[i]
            for k in xrange(len(kmer)-shortKmerLen+1):
                shortKmer = kmer[k:k+shortKmerLen]
                kmerListForTranscript.append(shortKmer)

        kmerList[transcriptID] = kmerListForTranscript

    createBinaryVectorFile(kmerList)
#    createHistogramFile(kmerList, histKmerLen)

fileName = "chr1_0_10mil_GID_TID_cDNA_strand.txt" 
baseCoverage = 5
longKmerLen = 100
shortKmerLen = 31
histKmerLen = 5
tlengs = callFasta(fileName, baseCoverage, longKmerLen, shortKmerLen, histKmerLen)

In [3]:
import re
import itertools
import numpy
import random

def loadFasta(filename):
    if (filename.endswith(".gz")):
        fp = gzip.open(filename, 'rb')
    else:
        fp = open(filename, 'rb')
    # split at headers
    data = fp.read().split('>')
    fp.close()
    # ignore whatever appears before the 1st header
    data.pop(0)     
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        # add an extra "+" to make string "1-referenced"
        sequences.append('+' + ''.join(lines))
    return (headers, sequences)

def createHistogramFile(kmerList, histKmerLen):
    histFile = open('kmerHistograms.csv', 'w')
    motifs = {}
    motifList = []
    
    for pattern in itertools.product('ACTG', repeat=histKmerLen):
        motif = ''.join(pattern)
        motifs[motif] = 0
        motifList.append(motif)
        histFile.write(motif)
        histFile.write(",")
    
    histFile.write("\n")
    freqKmers = {}
    
    for key, value in kmerList.iteritems():
        for kmer in value:
            for i in xrange(len(kmer)-histKmerLen+1):
                histKmer = kmer[i:i+histKmerLen]
#                 motifFreq =  motifs.get(histKmer, 0)
#                 motifFreq += 1
                motifs[histKmer] = motifs.get(histKmer, 0) + 1
        maxFreq = 0
        maxKmer = ''

        for motif in motifList:
            if motifs[motif] > maxFreq:
                maxFreq = motifs[motif]
                maxKmer = motif
            histFile.write(str(motifs[motif]))
            histFile.write(",")
            motifs[motif] = 0
        freqKmers[maxKmer] = freqKmers.get(maxKmer, 0)+1
            
        histFile.write(key)
        histFile.write("\n")

#    print freqKmers
    histFile.close()

def createBinaryVector(kmer):
    baseMatrices = {'A': '0001', 'C': '0010', 'G': '0100', 'T': '1000'}
    newKmer = ''
    for character in kmer:
        newKmer += baseMatrices.get(character)
    return newKmer

def callFasta(fileName, baseCoverage, longKmerLen, shortKmerLen, histKmerLen):
    header, seq = loadFasta(fileName)

    transcriptLocation = 1
    kmers = []
    transcripts = []
    validation_kmers = []
    validation_transcripts = []
    test_kmers = []
    test_transcripts = []
    
    for i in xrange(len(header)):
        transcriptID = re.split('\|+', header[i])[transcriptLocation][7:]
        for j in xrange(1, len(seq[i])-longKmerLen+1):
            kmer = seq[i][j:j+longKmerLen]
            for k in xrange(len(kmer)-shortKmerLen+1):
                shortKmer = createBinaryVector(kmer[k:k+shortKmerLen])
                kmers.append(shortKmer)
                transcripts.append(transcriptID)
                val = random.random()
                if val <= 0.20:
                    validation_kmers.append(shortKmer)
                    validation_transcripts.append(transcriptID)
                if val >= .80:
                    test_kmers.append(shortKmer)
                    test_transcripts.append(transcriptID)

        if len(seq[i])-1 <= longKmerLen:
            kmer = seq[i]
            for k in xrange(len(kmer)-shortKmerLen+1):
                shortKmer = createBinaryVector(kmer[k:k+shortKmerLen])
                kmers.append(shortKmer)
                transcripts.append(transcriptID)
                val = random.random()
                if val <= 0.20:
                    validation_kmers.append(shortKmer)
                    validation_transcripts.append(transcriptID)
                if val >= .80:
                    test_kmers.append(shortKmer)
                    test_transcripts.append(transcriptID)

    return kmers, transcripts, validation_kmers, validation_transcripts, test_kmers, test_transcripts

#    createHistogramFile(kmerList, histKmerLen)

fileName = "chr1_0_10mil_GID_TID_cDNA_strand_short.txt" 
baseCoverage = 5
longKmerLen = 100
shortKmerLen = 31
histKmerLen = 5
kmers, transcripts, validation_kmers, validation_transcripts, test_kmers, test_transcripts = callFasta(fileName, baseCoverage, longKmerLen, shortKmerLen, histKmerLen)
print len(kmers)
print len(transcripts)
print len(validation_kmers)
print len(validation_transcripts)
print len(test_kmers)
print len(test_transcripts)

0.394447004647
0.728162994921
0.84468014014


In [4]:
temp = {'AAATG': 2, 'GCCCA': 2, 'GCCCC': 1, 'AAATA': 1, 'AAATT': 1, 'TGTCT': 2, 'CAAAA': 1, 'CAAAG': 1, 'AGTAC': 1, 'CCGCC': 3, 'CCTAG': 1, 'TGGTG': 3, 'CGCAA': 1, 'GTGGT': 1, 'TCTGC': 2, 'GCTGT': 1, 'TCTGG': 2, 'CCCCA': 1, 'CCCCC': 1, 'AAGAA': 10, 'AAGAG': 1, 'AGCAG': 2, 'ACAGA': 1, 'CAGCA': 1, 'CTGGA': 2, 'CTGGC': 1, 'TCATC': 1, 'GAAAA': 6, 'TATAT': 1, 'AACAA': 2, 'TGAAG': 2, 'GAGTT': 1, 'CCTGG': 1, 'TGATG': 1, 'CCTTT': 1, 'GGCCG': 1, 'CCTTC': 1, 'CAAGA': 3, 'AGAAT': 1, 'AGAAG': 2, 'AGAAA': 9, 'TTAAA': 2, 'GCTTC': 1, 'AAGGA': 1, 'ATGCC': 1, 'AAGGC': 1, 'AATGG': 1, 'TTCCA': 1, 'GAAGA': 11, 'AATGA': 2, 'AGCCT': 2, 'CTCTG': 1, 'CCACA': 1, 'AATTC': 1, 'CTCCC': 1, 'GAGGA': 1, 'AAAGA': 1, 'TGCCA': 1, 'AATCC': 1, 'GGCAG': 1, 'GCGGC': 1, 'CCGGA': 1, 'AGAGG': 1, 'AGAGA': 4, 'GTGCA': 1, 'TGGAA': 3, 'TCTCC': 1, 'TGGAT': 2, 'AGCGG': 2, 'GTGTG': 3, 'ATGAT': 1, 'CTGCT': 1, 'TTTAA': 1, 'ATATA': 1, 'CTTCC': 1, 'ATATT': 1, 'CCTGA': 1, 'TCTTT': 1, 'TTTAT': 2, 'GGTGG': 2, 'ATTTT': 4, 'TGTGT': 1, 'CTCCT': 1, 'AAAAT': 4, 'CTGTG': 1, 'GACAC': 1, 'AAAAA': 28, 'AAAAC': 1, 'ATCAG': 1, 'ATCAA': 1, 'CTCTC': 3, 'TGAGA': 1, 'TGCTG': 8, 'AAACA': 1, 'GGCGG': 2, 'ACACA': 6, 'TTTTT': 33, 'AGGAA': 2, 'TAAAA': 2, 'TTTTA': 1, 'GATGA': 1, 'TGGGT': 1, 'ACAAC': 1, 'TTTCT': 1, 'ATGGA': 2, 'CAGAA': 3, 'GGAGG': 2, 'ATAGA': 1, 'TATTT': 2}

print len(temp)
#print [len(temp[i]) for i in xrange(len(temp)) if len(temp[i]) != 100]

109


In [2]:
import numpy
import theano
import theano.tensor as T

# x = T.ivector('input')
# y = T.argmax(x)
# get_argmax = theano.function([x], y)
# arrTest = numpy.array([1,2,3,4,19,1])
# print get_argmax(arrTest)

# elem = tensor.scalar('elem')
# elem = get_argmax(arrTest)
# val = -np.inf
# arrTest = tensor.set_subtensor(arrTest[elem:elem+1], val)

# print arrTest

W = theano.shared(
            value=numpy.random.rand(4, 4),
            name='W',
            borrow=True
        )

input = theano.shared(numpy.asarray([1,2,3,4],
                                dtype=theano.config.floatX),
                                borrow=True)

y_given_x = T.nnet.softmax(T.dot(input, W))

y_pred = T.argmax(y_given_x, axis=1)

print W.eval()
print input.eval()
print y_given_x
print y_given_x.ndim
print y_given_x.eval()
print y_pred.eval()

temp = y_given_x.eval()
temp[0,y_pred.eval()] = 0
y_given_x = theano.shared(temp)

y_pred = T.argmax(y_given_x, axis=1)

print y_given_x.eval()
print y_pred.eval()


# a = T.ivector('a')
# b = T.ivector('b')
# c = T.ivector('c')
# out1 = T.or_(T.or_(T.eq(a,b),T.eq(a,c)),T.eq(b,c))
#out2 = T.mean(T.eq(a,c))

# get_out1 = theano.function([a,b,c], out1)
#get_out2 = theano.function([a,c], out2)

# ans1 = get_out1([1,2,3,4], [7,8,9,10], [1,2,9,6])
#ans2 = get_out2([1,2,3,4], [1,2,5,6])

# print ans1
#print ans2

[[ 0.59785714  0.42324877  0.04288089  0.90853147]
 [ 0.11213601  0.82429483  0.97159137  0.19462965]
 [ 0.32879053  0.86143494  0.81872     0.98790726]
 [ 0.18781606  0.84905971  0.1761741   0.5781587 ]]
[ 1.  2.  3.  4.]
Softmax.0
2
[[ 0.00319925  0.77707296  0.04252415  0.17720363]]
[1]
[[ 0.00319925  0.          0.04252415  0.17720363]]
[3]


In [51]:
print type(y_given_x)
# print y_given_x.ndim
print temp.shape
print type(temp)

<class 'theano.tensor.var.TensorVariable'>
(1L, 4L)
<type 'numpy.ndarray'>
